# Methods to Detect Center

## Masking Low NDVI + Center of Mass

In [7]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        if true_color.max() > 0:
            true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color


# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 1 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode1 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            # Menyimpan data untuk menghubungkan koordinat piksel dengan koordinat dunia nyata
            # Yang disimpan:
            # Resolutsi citra: brp besar 1 piksel mewakili di dunia nyata dalam sumbu x dan y
            # Orientasi citra: diputar/miring
            # Posisi citra dalam sistem koordinat dunia nyata 
            transform = src.transform

        # # Perhitungan NDVI
        # NDVI (Normalized Difference Vegetation Index) adalah indeks yang digunakan untuk mendeteksi keberadaan vegetasi dengan membandingkan band NIR (Near Infrared) dan Red.
        # Rumus NDVI adalah: NDVI = (NIR - Red) / (NIR + Red)
        # Nilai NDVI berkisar antara -1 hingga 1, di mana nilai tinggi menunjukkan vegetasi yang lebat, dan nilai rendah (terutama mendekati -1) menunjukkan area tanpa vegetasi, seperti tanah atau air.
        # Fungsi ini digunakan untuk mendeteksi area vegetasi dan area yang mungkin merupakan pusat gunung berapi, di mana vegetasi lebih sedikit.

        with np.errstate(divide='ignore', invalid='ignore'):
            # Formula NDVI = (NIR - Red) / (NIR + Red)
            ndvi = (B8 - B4) / (B8 + B4)
            # Menghindari pembagian dengan 0 di tempat di mana (NIR + Red) sama dengan 0
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0

        # # Clipping nilai NDVI agar berada dalam rentang yang valid
        # Nilai NDVI yang lebih kecil dari -1 atau lebih besar dari 1 akan dipotong (clipped) ke dalam rentang tersebut.
        # Ini untuk menghindari data yang tidak sesuai atau anomali yang bisa terjadi selama perhitungan NDVI.
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])  # Menyaring nilai NaN
        valid_ndvi = flat_ndvi[mask_valid]

        # # KMeans Clustering
        # KMeans digunakan untuk mengelompokkan piksel berdasarkan nilai NDVI. Dengan menggunakan 2 cluster, kita ingin membedakan area dengan NDVI rendah (misalnya area tanpa vegetasi atau daerah kawah gunung berapi) dan NDVI tinggi (vegetasi).
        # Pengelompokan ini penting untuk menandai area dengan nilai NDVI rendah sebagai kandidat untuk pusat gunung berapi.
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah, yang biasanya mewakili area sekitar kawah atau puncak gunung berapi.
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)  # Cluster dengan nilai NDVI terendah
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster  # Masker untuk area NDVI rendah

        # # Menghitung Center of Mass
        # Center of mass adalah titik rata-rata dari distribusi massa pada objek dalam citra. Pada konteks ini, center of mass digunakan untuk menentukan titik pusat dari area NDVI rendah.
        # Setelah kita memiliki mask dengan cluster NDVI rendah, kita dapat menghitung titik pusat dari area tersebut menggunakan fungsi center_of_mass dari scipy.ndimage.
        # Ini dilakukan untuk mengetahui lokasi yang tepat dari pusat potensi gunung berapi berdasarkan citra NDVI.
        cy, cx = center_of_mass(ndvi_mask)
        cy, cx = int(cy), int(cx)  # Mengonversi koordinat ke integer

        # Mengonversi koordinat piksel (cy, cx) menjadi koordinat geospasial (latitude, longitude) berdasarkan transformasi citra
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi (latitude dan longitude) ke dalam dictionary untuk digunakan lebih lanjut
        hasil_deteksi_metode1[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Clustering KMeans
        axs[0, 1].imshow(labels.reshape(ndvi.shape), cmap='coolwarm')
        axs[0, 1].set_title('Hasil Clustering KMeans', fontsize=14)
        axs[0, 1].axis('off')

        # Mask Cluster NDVI Rendah
        axs[1, 0].imshow(ndvi_mask, cmap='gray')
        axs[1, 0].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # True Color
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')


        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()


# Menyimpan hasil deteksi ke dalam file CSV
import csv

csv_output_path = os.path.join(output_dir, 'hasil_deteksi_metode1.csv')

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['nama_gunung', 'koordinat_hasil_deteksi (lat, lon)'])
    for nama, data in hasil_deteksi_metode1.items():
        koordinat = f"{data['detected_lat']}, {data['detected_lon']}"
        writer.writerow([nama, koordinat])

## Masking Low NDVI + Largest Blob + Center of Mass

In [8]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
from skimage.measure import label, regionprops

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        if true_color.max() > 0:
            true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color

# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 2 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode2 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            transform = src.transform

        # Perhitungan NDVI
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (B8 - B4) / (B8 + B4)
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])
        valid_ndvi = flat_ndvi[mask_valid]

        # KMeans Clustering
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

        # Deteksi Largest Blob (komponen terbesar)
        labeled_mask = label(ndvi_mask)
        regions = regionprops(labeled_mask)
        largest_region = max(regions, key=lambda r: r.area)
        largest_blob_mask = labeled_mask == largest_region.label

        # Menghitung Center of Mass dari Blob Terbesar
        cy, cx = center_of_mass(largest_blob_mask)
        cy, cx = int(cy), int(cx)

        # Mengonversi koordinat piksel menjadi koordinat geospasial
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi
        hasil_deteksi_metode2[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        # Membuat visualisasi hasil deteksi
        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Mask Cluster NDVI Rendah
        axs[0, 1].imshow(ndvi_mask, cmap='gray')
        axs[0, 1].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[0, 1].axis('off')

        # Menambahkan legenda manual untuk mask NDVI rendah
        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[0, 1].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # Largest Blob
        axs[1, 0].imshow(largest_blob_mask, cmap='gray')
        axs[1, 0].set_title('Largest Blob (Area Terbesar)', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='Blob Terbesar'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)


        # True Color + Titik Pusat
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')

        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()

# Menyimpan hasil deteksi ke dalam file CSV
import csv

csv_output_path = os.path.join(output_dir, 'hasil_deteksi_metode2.csv')

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['nama_gunung', 'koordinat_deteksi (lat, lon)'])
    for nama, data in hasil_deteksi_metode1.items():
        koordinat = f"({data['detected_lat']}, {data['detected_lon']})"
        writer.writerow([nama, koordinat])


## Masking Low NDVI + Blob Closest to Center + Center of Mass

In [9]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
from skimage.measure import label, regionprops

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color

# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 3 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode3 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            transform = src.transform

        # Perhitungan NDVI
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (B8 - B4) / (B8 + B4)
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])
        valid_ndvi = flat_ndvi[mask_valid]

        # KMeans Clustering
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

        # Deteksi semua Blob (komponen terhubung)
        labeled_mask = label(ndvi_mask)
        regions = regionprops(labeled_mask)

        # Menentukan titik tengah gambar
        center_y, center_x = np.array(ndvi.shape) / 2

        # Cari blob yang paling dekat ke titik tengah gambar
        min_distance = np.inf
        closest_region = None
        for region in regions:
            # Menghitung center of mass masing-masing blob
            blob_cy, blob_cx = region.centroid
            distance = np.sqrt((blob_cy - center_y) ** 2 + (blob_cx - center_x) ** 2)
            if distance < min_distance:
                min_distance = distance
                closest_region = region

        # Membuat mask untuk blob terdekat
        closest_blob_mask = labeled_mask == closest_region.label

        # Menghitung Center of Mass dari Blob Terdekat
        cy, cx = center_of_mass(closest_blob_mask)
        cy, cx = int(cy), int(cx)

        # Mengonversi koordinat piksel menjadi koordinat geospasial
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi
        hasil_deteksi_metode3[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        # Membuat visualisasi hasil deteksi
        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Mask Cluster NDVI Rendah
        axs[0, 1].imshow(ndvi_mask, cmap='gray')
        axs[0, 1].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[0, 1].axis('off')

        # Menambahkan legenda manual untuk mask NDVI rendah
        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[0, 1].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # Blob Closest to Center
        axs[1, 0].imshow(closest_blob_mask, cmap='gray')
        axs[1, 0].set_title('Blob Closest to Center', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='Blob Terdekat ke Pusat'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # True Color + Titik Pusat
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')

        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()

# Menampilkan hasil deteksi
print("Hasil Deteksi Pusat Gunung:")
for nama, data in hasil_deteksi_metode3.items():
    print(f"{nama}: {data['detected_lat']}, {data['detected_lon']}")


Hasil Deteksi Pusat Gunung:
Agung: -8.339195529774136, 115.5071309468921
Ambang: 0.7466347483959402, 124.41976603828394
Anak_Krakatau: -6.0994260318789335, 105.42506901863348
Anak_Ranakah: -8.636897214931345, 120.53015068957482
Arjuno_Welirang: -7.752595649244089, 112.58473166459447
Awu: 3.6860121895634266, 125.45166080515203
Banda_Api: -4.522433550607112, 129.88026532433287
Batur: -8.237506239611806, 115.3770548937516
Batutara: -7.7932893316147025, 123.58873456894497
Bromo: -7.9482487181253205, 112.95465789859489
Bur_Ni_Telong: 4.768392275399038, 96.81974758593893
Ciremai: -6.906023325489851, 108.4053198052284
Colo: -0.17126380891738677, 121.60525442160906
Dempo: -4.015244741193231, 103.1216089670942
Dieng: -7.207048777198303, 109.88035350175267
Dukono: 1.700735411659284, 127.87720190380315
Ebulobo: -8.814494146601774, 121.19122090715838
Egon: -8.673189152409774, 122.4570369740112
Galunggung: -7.232650762795709, 108.0640498287914
Gamalama: 0.8080795138297155, 127.32797193909248
Gamkon

## Integrasi DEM untuk Metode 1

In [10]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
import csv

def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)
        if true_color.max() > 0:
            true_color = true_color / true_color.max()
        return true_color

# Folder input dan output
sentinel_dir = '../DATA/SENTINEL 2/'
dem_dir = '../DATA/DEM/'
output_dir = 'Metode 1 Integrasi DEM'
os.makedirs(output_dir, exist_ok=True)

# Daftar 4 gunung dengan buffer
volcanoes = ["Anak_Krakatau", "Wurlali", "Banda_Api", "Colo"]
hasil_deteksi = {}

for name in volcanoes:
    sentinel_path = os.path.join(sentinel_dir, f'Sentinel2_{name}.tif')
    dem_path = os.path.join(dem_dir, f'DEM_SRTM_{name}.tif')

    if not os.path.exists(sentinel_path) or not os.path.exists(dem_path):
        print(f"File tidak ditemukan untuk {name}, lewati...")
        continue

    with rasterio.open(sentinel_path) as src:
        B4 = src.read(4).astype('float32')
        B8 = src.read(8).astype('float32')
        transform = src.transform
        crs = src.crs
        width, height = src.width, src.height

    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (B8 - B4) / (B8 + B4)
        ndvi[(B8 + B4) == 0] = np.nan
    ndvi = np.clip(ndvi, -1, 1)

    # Baca DEM dan resample jika ukurannya beda
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1).astype('float32')
        if dem_data.shape != ndvi.shape:
            from rasterio.warp import reproject, Resampling
            dem_resampled = np.empty_like(ndvi)
            reproject(
                source=dem_data,
                destination=dem_resampled,
                src_transform=dem_src.transform,
                src_crs=dem_src.crs,
                dst_transform=transform,
                dst_crs=crs,
                resampling=Resampling.bilinear
            )
        else:
            dem_resampled = dem_data

    # Gabung NDVI dan DEM
    flat_ndvi = ndvi.reshape(-1)
    flat_dem = dem_resampled.reshape(-1)
    mask_valid = ~np.isnan(flat_ndvi) & ~np.isnan(flat_dem)
    features = np.stack([flat_ndvi[mask_valid], flat_dem[mask_valid]], axis=1)

    # KMeans clustering
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = np.zeros_like(flat_ndvi, dtype=np.uint8)
    labels[mask_valid] = kmeans.fit_predict(features)

    cluster_means = [features[labels[mask_valid] == i][:, 0].mean() for i in range(2)]
    low_ndvi_cluster = np.argmin(cluster_means)
    ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

    # Center of Mass
    cy, cx = center_of_mass(ndvi_mask)
    cy, cx = int(cy), int(cx)
    lon, lat = rasterio.transform.xy(transform, cy, cx)
    hasil_deteksi[name] = {'lat': lat, 'lon': lon}

    # Visualisasi
    true_color = create_true_color_image(sentinel_path)
    fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)
    axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    axs[0, 0].set_title('NDVI Map'); axs[0, 0].axis('off')
    axs[0, 1].imshow(labels.reshape(ndvi.shape), cmap='coolwarm')
    axs[0, 1].set_title('KMeans Clustering'); axs[0, 1].axis('off')
    axs[1, 0].imshow(ndvi_mask, cmap='gray')
    axs[1, 0].set_title('NDVI Rendah Mask'); axs[1, 0].axis('off')
    axs[1, 1].imshow(true_color)
    axs[1, 1].scatter(cx, cy, c='red', s=50, label='Titik Pusat')
    axs[1, 1].legend(); axs[1, 1].set_title('True Color + Titik Pusat'); axs[1, 1].axis('off')
    fig.suptitle(f'{name} | Deteksi Pusat\nKoordinat: {lat:.5f}, {lon:.5f}', fontsize=16)
    plt.savefig(os.path.join(output_dir, f'{name}.png'))
    plt.close()

# Simpan ke CSV
csv_path = os.path.join(output_dir, 'hasil_deteksi_metode1_integrasi_dem.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['nama_gunung', 'lat', 'lon'])
    for name, coord in hasil_deteksi.items():
        writer.writerow([name, coord['lat'], coord['lon']])


# Evaluation (Calculate Distance to KRB Center Point)

## Import KRB Center Point Data

In [8]:
import pandas as pd

# Membaca CSV
csv_path = 'List KRB center point.csv'
df_krb = pd.read_csv(csv_path)

# Membuat dictionary dari data CSV
krb_center_points = {}

for idx, row in df_krb.iterrows():
    nama = row['Nama']
    lat_lon_str = row['Titik Koordinat Pusat']
    lat_str, lon_str = lat_lon_str.split(',')
    lat, lon = float(lat_str.strip()), float(lon_str.strip())

    krb_center_points[nama] = {
        'detected_lat': lat,
        'detected_lon': lon
    }

# Contoh output
print("Dictionary Koordinat KRB:")
for nama, koordinat in krb_center_points.items():
    print(f"{nama}: {koordinat['detected_lat']}, {koordinat['detected_lon']}")


Dictionary Koordinat KRB:
Agung: -8.341953075466813, 115.50750732421875
Ambang: 0.7535717180105911, 124.42119598388675
Anak Krakatau: -6.1016391293547425, 105.4241180419922
Anak Ranakah: -8.633806608897267, 120.53272247314456
Arjuno Welirang: -7.733445463620818, 112.57484436035155
Awu: 3.682345501935164, 125.4521942138672
Banda Api: -4.52269310446871, 129.88105773925784
Batur: -8.24563904165236, 115.36931991577148
Batutara: -7.789914903590686, 123.58777999877933
Bromo: -7.941596147894652, 112.95181274414064
Bur Ni Telong: 4.768730969283216, 96.82113647460939
Ciremai: -6.895752309876321, 108.4079360961914
Colo: -0.16891455023785007, 121.60766601562503
Dempo: -4.01564458761465, 103.11973571777345
Dieng: -7.199681955820926, 109.84199523925783
Dukono: 1.700571185447282, 127.87742614746094
Ebulobo: -8.815698116551511, 121.19104385375978
Egon: -8.677421123289987, 122.45412826538086
Galunggung: -7.256561216674466, 108.07594299316408
Gamalama: 0.8084982528736103, 127.32982635498048
Gamkonora: 

## Calculate Distance for All Methods

In [9]:
import numpy as np
import pandas as pd

# Fungsi untuk menghitung jarak Haversine (kilometer)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius bumi dalam kilometer
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# List hasil semua
hasil_perbandingan = []

for nama_krb in krb_center_points.keys():
    nama_lookup = nama_krb.replace('Laki-laki','Laki_Laki').replace(' ', '_')  # hanya untuk lookup ke hasil_deteksi

    if nama_lookup in hasil_deteksi_metode1 and nama_lookup in hasil_deteksi_metode2 and nama_lookup in hasil_deteksi_metode3:
        lat_krb = krb_center_points[nama_krb]['detected_lat']
        lon_krb = krb_center_points[nama_krb]['detected_lon']

        jarak1 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode1[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode1[nama_lookup]['detected_lon'])
        jarak2 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode2[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode2[nama_lookup]['detected_lon'])
        jarak3 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode3[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode3[nama_lookup]['detected_lon'])

        jarak1 = round(jarak1, 2)
        jarak2 = round(jarak2, 2)
        jarak3 = round(jarak3, 2)

        hasil_perbandingan.append({
            'Gunung': nama_krb,   # pakai nama asli dari KRB (tetap spasi)
            'M1': jarak1,
            'M2': jarak2,
            'M3': jarak3
        })

# Buat DataFrame
df_perbandingan = pd.DataFrame(hasil_perbandingan)

# Fungsi pilih metode terbaik dan terburuk
def pilih_metode(row):
    values = {
        'Metode 1': row['M1'],
        'Metode 2': row['M2'],
        'Metode 3': row['M3']
    }
    metode_terbaik = min(values, key=values.get)
    metode_terburuk = max(values, key=values.get)
    jarak_min = round(min(values.values()), 2)
    jarak_max = round(max(values.values()), 2)
    return pd.Series([jarak_min, metode_terbaik, jarak_max, metode_terburuk],
                     index=['Min', 'Best Method', 'Max', 'Worst Method'])

# Apply ke DataFrame
df_perbandingan[['Min', 'Best Method', 'Max', 'Worst Method']] = df_perbandingan.apply(pilih_metode, axis=1)

# Print hasil
print(df_perbandingan)

# (Opsional) Simpan ke CSV
df_perbandingan.to_csv('(NEW)hasil_perbandingan_fix.csv', index=False)


              Gunung    M1    M2    M3   Min Best Method   Max Worst Method
0              Agung  0.46  0.31  0.31  0.31    Metode 2  0.46     Metode 1
1             Ambang  0.64  0.45  0.79  0.45    Metode 2  0.79     Metode 3
2      Anak Krakatau  0.17  0.40  0.27  0.17    Metode 1  0.40     Metode 2
3       Anak Ranakah  1.10  0.29  0.45  0.29    Metode 2  1.10     Metode 1
4    Arjuno Welirang  2.39  3.48  2.39  2.39    Metode 1  3.48     Metode 2
..               ...   ...   ...   ...   ...         ...   ...          ...
63           Tambora  2.47  2.64  2.91  2.47    Metode 1  2.91     Metode 3
64          Tandikat  4.23  5.02  4.39  4.23    Metode 1  5.02     Metode 2
65          Tangkoko  3.51  4.31  3.14  3.14    Metode 3  4.31     Metode 2
66  Tangkuban Parahu  5.80  8.80  2.07  2.07    Metode 3  8.80     Metode 2
67           Wurlali  0.12  0.10  0.31  0.10    Metode 2  0.31     Metode 3

[68 rows x 8 columns]


## Final Visualization (True Color with 3 Points from Each Methods + 1 KRB Center Point)

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Folder input dan output
input_dir = '../../DATA/SENTINEL v6/'
output_dir_viz = 'final'
os.makedirs(output_dir_viz, exist_ok=True)

# Offset kecil untuk menghindari titik bertumpuk
offsets = {
    'krb': (0, 0),
    'metode1': (2, 2),
    'metode2': (-2, 2),
    'metode3': (2, -2)
}

# Loop per file citra
for nama_krb in krb_center_points.keys():
    # Modify this line to handle both hyphen '-' and space ' '
    nama_lookup = nama_krb.replace('Laki-laki', 'Laki_Laki').replace(' ', '_').replace('-', '_')


    if nama_lookup in hasil_deteksi_metode1 and nama_lookup in hasil_deteksi_metode2 and nama_lookup in hasil_deteksi_metode3:
        filepath = os.path.join(input_dir, f'Sentinel2_{nama_lookup}.tif')

        if os.path.exists(filepath):
            with rasterio.open(filepath) as src:
                band4 = src.read(4).astype('float32')  # Red
                band3 = src.read(3).astype('float32')  # Green
                band2 = src.read(2).astype('float32')  # Blue

                # Buat true color
                true_color = np.dstack((band4, band3, band2))
                true_color = np.nan_to_num(true_color, nan=0)
                if true_color.max() > 0:
                    true_color = true_color / true_color.max()

                # Konversi koordinat ke piksel
                def geo_to_pixel(lon, lat):
                    row, col = src.index(lon, lat)
                    return col, row  # Note: col = x, row = y di gambar

                # Ambil titik
                lat_krb, lon_krb = krb_center_points[nama_krb]['detected_lat'], krb_center_points[nama_krb]['detected_lon']
                lat_m1, lon_m1 = hasil_deteksi_metode1[nama_lookup]['detected_lat'], hasil_deteksi_metode1[nama_lookup]['detected_lon']
                lat_m2, lon_m2 = hasil_deteksi_metode2[nama_lookup]['detected_lat'], hasil_deteksi_metode2[nama_lookup]['detected_lon']
                lat_m3, lon_m3 = hasil_deteksi_metode3[nama_lookup]['detected_lat'], hasil_deteksi_metode3[nama_lookup]['detected_lon']

                x_krb, y_krb = geo_to_pixel(lon_krb, lat_krb)
                x_m1, y_m1 = geo_to_pixel(lon_m1, lat_m1)
                x_m2, y_m2 = geo_to_pixel(lon_m2, lat_m2)
                x_m3, y_m3 = geo_to_pixel(lon_m3, lat_m3)

                # Plot
                plt.figure(figsize=(10, 10))
                plt.imshow(true_color)

                plt.scatter(x_krb + offsets['krb'][0], y_krb + offsets['krb'][1], c='cyan', s=50, label='Titik KRB')
                plt.scatter(x_m1 + offsets['metode1'][0], y_m1 + offsets['metode1'][1], c='red', s=50, label='Metode 1')
                plt.scatter(x_m2 + offsets['metode2'][0], y_m2 + offsets['metode2'][1], c='pink', s=50, label='Metode 2')
                plt.scatter(x_m3 + offsets['metode3'][0], y_m3 + offsets['metode3'][1], c='yellow', s=50, label='Metode 3')

                plt.legend(loc='lower right')
                plt.title(f'Visualisasi Hasil Deteksi Pusat Gunung {nama_krb}', fontsize=16, fontweight='bold')
                plt.axis('off')

                # Save
                output_path = os.path.join(output_dir_viz, f'{nama_krb}.png')
                plt.savefig(output_path, bbox_inches='tight')
                plt.close()

print("Semua visualisasi titik telah selesai disimpan!")


Semua visualisasi titik telah selesai disimpan!


## Mengecek apakah ada titik KRB yang tidak termasuk dalam citra

In [ ]:
import rasterio
import os

# Folder input
input_dir = '../../DATA/NEW AREA/'

# Untuk menyimpan hasil
krb_outside_image = []

for nama_krb in krb_center_points.keys():
    nama_lookup = nama_krb.replace(' ', '_').replace('-', '_')

    filepath = os.path.join(input_dir, f'Sentinel2_{nama_lookup}.tif')

    if os.path.exists(filepath):
        with rasterio.open(filepath) as src:
            bounds = src.bounds  # Bounding box citra: left, bottom, right, top

        lat_krb = krb_center_points[nama_krb]['detected_lat']
        lon_krb = krb_center_points[nama_krb]['detected_lon']

        # Cek apakah lon dan lat KRB masuk dalam bounding box
        if not (bounds.left <= lon_krb <= bounds.right and bounds.bottom <= lat_krb <= bounds.top):
            krb_outside_image.append(nama_krb)

# Tampilkan hasil
if krb_outside_image:
    print("Titik KRB berikut berada di luar area citra:")
    for nama in krb_outside_image:
        print("-", nama)
else:
    print("Semua titik KRB berada dalam area citra.")


Titik KRB berikut berada di luar area citra:
- Dieng
- Tandikat
- Tangkoko
